### Linear and Multilinear Functions

In [1]:
from kingdon import Algebra
from gc_utils import *
import numpy as np
# use vanilla algebra, so we're not bother by the sign of I*I.reverse()
alg = Algebra(4) 
locals().update(alg.blades)

In [2]:
#1.1 linear transformation
A = create_random_multivector(alg)
B = create_random_multivector(alg)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
x = create_random_multivector(alg).grade(1)
a = create_random_multivector(alg).grade(1)
f = lambda x: (A*x*B+C*x*D).grade(1)
differential(f, x, a), f(a)

(0.747 𝐞₁ + 0.492 𝐞₂ + -0.198 𝐞₃ + 1.26 𝐞₄,
 0.747 𝐞₁ + 0.492 𝐞₂ + -0.198 𝐞₃ + 1.26 𝐞₄)

In [3]:
#1.2
g = lambda x: (C*x*D).grade(1)
g(f(a)), differential(g, f(a), f(a))

(-1.66 𝐞₁ + 1.44 𝐞₂ + 5.71 𝐞₃ + -1.13 𝐞₄,
 -1.66 𝐞₁ + 1.44 𝐞₂ + 5.71 𝐞₃ + -1.13 𝐞₄)

In [4]:
# modify the input of the original skew_symmetrizer: a multivector Ar instead of vectors
def skew_symmetrizer(F, Ar, alg, h=1e-6, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(alg.frame)
    drF = 0
    r = Ar.grades[0]
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        # if F is linear, vectors cause no difference in vectors_partial. The input is just Ar.
        # So actually we allow Ar being any multivector
        drF += (Ar.sp(wedge(base_vectors[::-1]))) * vectors_partial(F, np.zeros(r), reci_vectors, h=h)
    return (1/factorial(r)) * drF

In [5]:
#1.4 Extend a vector function to a multivector function.
# For linear f we ignore the position where f is measured, 
# just the differential at each direction.
def outermorphism(f, A: MultiVector, alg, h=1e-6, frame=None, r_frame=None):
    wf = lambda vectors: wedge([f(v) for v in vectors])
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
    return outer

outermorphism(f, x, alg), f(x)

(0.907 𝐞₁ + 1.41 𝐞₂ + 3.85 𝐞₃ + 3.74 𝐞₄,
 0.907 𝐞₁ + 1.41 𝐞₂ + 3.85 𝐞₃ + 3.74 𝐞₄)

In [6]:
#1.6 A frame can be arbitrary as long as it represent the same space. 
# It's pretty handy to specify a domain.
h = 1e-2
A2 = e12
A2frame = [e1, e2]
A2r_frame = reciprocal(A2frame)

outermorphism(f, P(A, A2), alg, h, frame=A2frame, r_frame=A2r_frame), outermorphism(f, A, alg, h, frame=A2frame, r_frame=A2r_frame)

(0.371 + 0.17 𝐞₁ + -0.305 𝐞₂ + 0.732 𝐞₃ + 0.425 𝐞₄ + 1.5 𝐞₁₂ + -12.8 𝐞₁₃ + -4.9 𝐞₁₄ + 16.5 𝐞₂₃ + 5.05 𝐞₂₄ + 10.9 𝐞₃₄,
 0.371 + 0.17 𝐞₁ + -0.305 𝐞₂ + 0.732 𝐞₃ + 0.425 𝐞₄ + 1.5 𝐞₁₂ + -12.8 𝐞₁₃ + -4.9 𝐞₁₄ + 16.5 𝐞₂₃ + 5.05 𝐞₂₄ + 10.9 𝐞₃₄)

In [7]:
#1.7 An outermorphism preserves outer products
vectors = create_r_vectors(3, alg)
outermorphism(f, wedge(vectors), alg), wedge([f(v) for v in vectors])

(24.8 𝐞₁₂₃ + 10.8 𝐞₁₂₄ + -11.8 𝐞₁₃₄ + 35.8 𝐞₂₃₄,
 24.8 𝐞₁₂₃ + 10.8 𝐞₁₂₄ + -11.8 𝐞₁₃₄ + 35.8 𝐞₂₃₄)

In [8]:
# By 1.7, splitting A into simple blades, we get an easier implement of 1.4
# Here, the value at v replaces the slope at direction v by linearity
def outermorphism_(f, A: MultiVector, alg):
    outer = A.grade(0)
    for vectors in multi_frame_vectors(alg.frame):
        Vr = wedge(reciprocal(vectors)[::-1])
        outer += alg.sp(Vr,A) * wedge([f(v) for v in vectors]) # frame split of A
    return outer

outermorphism_(f, D, alg), outermorphism(f, D, alg)

(0.259 + 0.0469 𝐞₁ + 1.86 𝐞₂ + 4.99 𝐞₃ + 3.14 𝐞₄ + 4.92 𝐞₁₂ + -3.69 𝐞₁₃ + 0.691 𝐞₁₄ + 9.08 𝐞₂₃ + -3.55 𝐞₂₄ + 0.753 𝐞₃₄ + 65.9 𝐞₁₂₃ + 29.3 𝐞₁₂₄ + -31.8 𝐞₁₃₄ + 98.9 𝐞₂₃₄ + -4.71 𝐞₁₂₃₄,
 0.259 + 0.0469 𝐞₁ + 1.86 𝐞₂ + 4.99 𝐞₃ + 3.14 𝐞₄ + 4.92 𝐞₁₂ + -3.69 𝐞₁₃ + 0.691 𝐞₁₄ + 9.08 𝐞₂₃ + -3.55 𝐞₂₄ + 0.753 𝐞₃₄ + 65.9 𝐞₁₂₃ + 29.3 𝐞₁₂₄ + -31.8 𝐞₁₃₄ + 98.9 𝐞₂₃₄ + -4.71 𝐞₁₂₃₄)

In [9]:
#1.9 anoter outer product property
outermorphism(f, A.reverse(), alg), outermorphism(f, A, alg).reverse()

(0.371 + 0.475 𝐞₁ + 1.98 𝐞₂ + 1.24 𝐞₃ + 2.16 𝐞₄ + -16.9 𝐞₁₂ + 26.0 𝐞₁₃ + 3.25 𝐞₁₄ + -19.7 𝐞₂₃ + 16.7 𝐞₂₄ + -29.0 𝐞₃₄ + -28.9 𝐞₁₂₃ + -13.3 𝐞₁₂₄ + 13.8 𝐞₁₃₄ + -43.8 𝐞₂₃₄ + -7.03 𝐞₁₂₃₄,
 0.371 + 0.475 𝐞₁ + 1.98 𝐞₂ + 1.24 𝐞₃ + 2.16 𝐞₄ + -16.9 𝐞₁₂ + 26.0 𝐞₁₃ + 3.25 𝐞₁₄ + -19.7 𝐞₂₃ + 16.7 𝐞₂₄ + -29.0 𝐞₃₄ + -28.9 𝐞₁₂₃ + -13.3 𝐞₁₂₄ + 13.8 𝐞₁₃₄ + -43.8 𝐞₂₃₄ + -7.03 𝐞₁₂₃₄)

In [10]:
#1.10
fA2 = lambda x: P(f(x), A2)
outermorphism(fA2, A, alg), P(outermorphism(fA2, A, alg), A2)

(0.371 + 0.475 𝐞₁ + 1.98 𝐞₂ + 16.9 𝐞₁₂, 0.371 + 0.475 𝐞₁ + 1.98 𝐞₂ + 16.9 𝐞₁₂)

In [11]:
#1.11
def adjoint_outermorphism(f, A, alg, h=1e-6, frame=None, r_frame=None):
    F = lambda vectors: wedge([f(v) for v in vectors]).sp(A)
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        # Why np.zeros(r)? The derivative of a linear function is constant
        outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
    return outer

In [12]:
#1.12
adjoint_outermorphism(fA2, P(A, A2), alg), adjoint_outermorphism(fA2, A, alg)

(0.371 + 0.289 𝐞₁ + -0.347 𝐞₂ + 0.526 𝐞₃ + 0.0829 𝐞₄ + 1.5 𝐞₁₂ + -12.8 𝐞₁₃ + 1.8 𝐞₁₄ + 12.6 𝐞₂₃ + -2.6 𝐞₂₄ + 6.94 𝐞₃₄,
 0.371 + 0.289 𝐞₁ + -0.347 𝐞₂ + 0.526 𝐞₃ + 0.0829 𝐞₄ + 1.5 𝐞₁₂ + -12.8 𝐞₁₃ + 1.8 𝐞₁₄ + 12.6 𝐞₂₃ + -2.6 𝐞₂₄ + 6.94 𝐞₃₄)

In [13]:
#1.13
# Exercise: implement the adjoint outermorphism by spliting A into vectors.
adjoint_outermorphism(f, A^B, alg), adjoint_outermorphism(f, A, alg) ^ adjoint_outermorphism(f, B, alg)

(0.0202 + -0.00185 𝐞₁ + 1.68 𝐞₂ + 2.69 𝐞₃ + -0.168 𝐞₄ + 11.2 𝐞₁₂ + 4.82 𝐞₁₃ + 0.608 𝐞₁₄ + 19.8 𝐞₂₃ + -10.0 𝐞₂₄ + -5.31 𝐞₃₄ + 2.88e+02 𝐞₁₂₃ + -38.1 𝐞₁₂₄ + -32.0 𝐞₁₃₄ + 1.92e+02 𝐞₂₃₄ + -11.4 𝐞₁₂₃₄,
 0.0202 + -0.00185 𝐞₁ + 1.68 𝐞₂ + 2.69 𝐞₃ + -0.168 𝐞₄ + 11.2 𝐞₁₂ + 4.82 𝐞₁₃ + 0.608 𝐞₁₄ + 19.8 𝐞₂₃ + -10.0 𝐞₂₄ + -5.31 𝐞₃₄ + 2.88e+02 𝐞₁₂₃ + -38.1 𝐞₁₂₄ + -32.0 𝐞₁₃₄ + 1.92e+02 𝐞₂₃₄ + -11.4 𝐞₁₂₃₄)

In [14]:
#1.14a r <= s
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
    adjoint_outermorphism(f, outermorphism(f, Ar, alg) | Bs, alg),
)

(-34.5 𝐞₁ + -13.7 𝐞₂ + -35.9 𝐞₃ + -19.7 𝐞₄,
 -5.6 𝐞₁ + -11.3 𝐞₂ + -7.16 𝐞₃ + -2.64 𝐞₄,
 -34.5 𝐞₁ + -13.7 𝐞₂ + -35.9 𝐞₃ + -19.7 𝐞₄)

In [15]:
#1.14b r >= s
r, s = 3, 2
Ar = A.grade(r)
Bs = B.grade(s)
(
    outermorphism(f, Ar | adjoint_outermorphism(f, Bs, alg), alg),
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
)

(-21.5 𝐞₁ + -6.76 𝐞₂ + 19.0 𝐞₃ + -27.0 𝐞₄,
 1.44 𝐞₁ + -0.337 𝐞₂ + -5.14 𝐞₃ + 0.354 𝐞₄,
 -21.5 𝐞₁ + -6.76 𝐞₂ + 19.0 𝐞₃ + -27.0 𝐞₄)

In [16]:
#1.15 r = s
for r,s in [(1,1), (2,2), (3,3)]:
    Ar = A.grade(r)
    Bs = B.grade(s)
    print(
        Ar | adjoint_outermorphism(f, Bs, alg),
        outermorphism(f, Ar, alg) | Bs,
    )

3.84 3.84
-9.33 -9.33
-14.5 -14.5


In [17]:
# proof of 1.14a
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
frame = alg.frame
r_frame = reciprocal(alg.frame)
rhs = 0
r = Ar.grades[0]
F = lambda vectors: wedge([f(v) for v in vectors]) 
for base_vectors, reci_vectors in zip(permutations(frame, s), permutations(r_frame, s)):
    rhs += (Ar | (wedge(base_vectors[::-1]))) * (vectors_partial(F, np.zeros(s), reci_vectors, h=h) | Bs)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    rhs / factorial(s)
)

(-34.5 𝐞₁ + -13.7 𝐞₂ + -35.9 𝐞₃ + -19.7 𝐞₄,
 -34.5 𝐞₁ + -13.7 𝐞₂ + -35.9 𝐞₃ + -19.7 𝐞₄)

In [18]:
# singular, nonsingular functions
I = alg.pseudoscalar((1,))
# fA2 is singular
outermorphism(fA2, I, alg), outermorphism(f, I, alg), adjoint_outermorphism(f, I, alg)

(, -8.24 𝐞₁₂₃₄, -8.24 𝐞₁₂₃₄)

In [19]:
#1.16 Use inverse instead of reverse for negative signatures. case I = Ip
n = 4
Ir = I.inv()
F = lambda vectors: wedge([f(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame, alg, h=1e-2), 
    Ir * outermorphism(f, I, alg), 
    adjoint_outermorphism(f, I, alg) * Ir
)

(-8.24, -8.24, -8.24)

In [20]:
#1.16 case I != Ip
n = 2
I = e12
Ip = e23
Ir = I.inv()
Ipr = Ip.inv()
h = 1e-2
# a function from I to Ip
g = lambda x: P((A*P(x, I)*B).grade(1), Ip)
F = lambda vectors: wedge([g(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame[:2], alg, h),
    Ir * outermorphism(g, I, alg, h),
    adjoint_outermorphism(g, Ip, alg, h) * Ipr
)

(-1.83 𝐞₁₃, -1.83 𝐞₁₃, -1.83 𝐞₁₃)

In [21]:
#1.17a
outermorphism(g, I, alg, h), Ip

(1.83 𝐞₂₃, 1 𝐞₂₃)

In [22]:
#1.17b Need extra minus sign for negative signature. You can try Algebra(3,1).
adjoint_outermorphism(g, Ip, alg, h), I

(1.83 𝐞₁₂, 1 𝐞₁₂)

In [23]:
# The relative orientation between I and I' can be arbitrary.
# Here we let the parent algebra determine the orientation.
def det(f, I, Ip):
    return (outermorphism(f, I, alg)/Ip)[0]

det(g, I, Ip)

1.8302149969789572

In [24]:
#1.21a null part goes null
C = create_random_multivector(alg)
C_ = outermorphism(g, C, alg)
(1/det(g, I, Ip)) * adjoint_outermorphism(g, C_*Ip, alg) * Ir, C

(0.778 + 0.934 𝐞₁ + 0.719 𝐞₂ + 0.773 𝐞₁₂,
 0.778 + 0.934 𝐞₁ + 0.719 𝐞₂ + 0.923 𝐞₃ + 0.889 𝐞₄ + 0.773 𝐞₁₂ + 0.798 𝐞₁₃ + 0.438 𝐞₁₄ + 0.912 𝐞₂₃ + 0.886 𝐞₂₄ + 0.806 𝐞₃₄ + 0.402 𝐞₁₂₃ + 0.982 𝐞₁₂₄ + 0.0427 𝐞₁₃₄ + 0.788 𝐞₂₃₄ + 0.619 𝐞₁₂₃₄)

In [25]:
#1.21b null part goes null
C_ = create_random_multivector(alg)
C = adjoint_outermorphism(g, C_, alg)
(1/det(g, I, Ip)) * outermorphism(g, C*I, alg) * Ipr, C_

(0.868 + 0.774 𝐞₂ + 0.605 𝐞₃ + 0.0746 𝐞₂₃,
 0.868 + 0.588 𝐞₁ + 0.774 𝐞₂ + 0.605 𝐞₃ + 0.249 𝐞₄ + 0.706 𝐞₁₂ + 0.00499 𝐞₁₃ + 0.424 𝐞₁₄ + 0.0746 𝐞₂₃ + 0.199 𝐞₂₄ + 0.334 𝐞₃₄ + 0.636 𝐞₁₂₃ + 0.0659 𝐞₁₂₄ + 0.478 𝐞₁₃₄ + 0.351 𝐞₂₃₄ + 0.319 𝐞₁₂₃₄)

In [26]:
#2.1 Note that characteristic multivectors are of even grades
r = 3  # try several r
h = 1e-2
vectors = create_r_vectors(r, alg)
F = lambda vectors: wedge([f(v) for v in vectors])
(
    F(vectors),
    outermorphism(f, wedge(vectors), alg, h),
    simplicial_derivative(F, vectors, alg, h), # characteristic multivectors
    derivative(lambda A: outermorphism(f, A, alg, h), wedge(vectors), alg, h, r),
)


(-5.39 𝐞₁₂₃ + -2.5 𝐞₁₂₄ + 6.97 𝐞₁₃₄ + -8.34 𝐞₂₃₄,
 -5.39 𝐞₁₂₃ + -2.5 𝐞₁₂₄ + 6.97 𝐞₁₃₄ + -8.34 𝐞₂₃₄,
 7.82 + 2.79 𝐞₁₂ + 10.7 𝐞₁₃ + 11.1 𝐞₁₄ + 8.69 𝐞₂₃ + 11.0 𝐞₂₄ + 10.2 𝐞₃₄,
 7.82 + 2.79 𝐞₁₂ + 10.7 𝐞₁₃ + 11.1 𝐞₁₄ + 8.69 𝐞₂₃ + 11.0 𝐞₂₄ + 10.2 𝐞₃₄)

In [27]:
#2.2
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
images = [f(v) for v in frame]
(
    sum(vr * b for b, vr in zip(images, r_frame)),
    derivative(f, x, alg)
)

(-3.99 + 1.32 𝐞₁₂ + 0.47 𝐞₁₃ + 2.61 𝐞₁₄ + 0.679 𝐞₂₃ + 1.39 𝐞₂₄ + 1.01 𝐞₃₄,
 -3.99 + -0.0805 𝐞₁ + 3.57 𝐞₂ + 2.62 𝐞₃ + 5.75 𝐞₄ + 4.19 𝐞₁₂ + 0.203 𝐞₁₃ + 5.9 𝐞₁₄ + 2.07 𝐞₂₃ + 1.95 𝐞₂₄ + 3.37 𝐞₃₄ + 3.5 𝐞₁₂₃ + -0.571 𝐞₁₂₄ + 3.17 𝐞₁₃₄ + -1.51 𝐞₂₃₄ + 1.61 𝐞₁₂₃₄)

In [28]:
#2.2 comes from linearity
images, [differential(f, a, v) for v in frame]

([-0.916 𝐞₁ + 0.985 𝐞₂ + 2.27 𝐞₃ + 0.959 𝐞₄,
  -0.0284 𝐞₁ + 0.354 𝐞₂ + 2.41 𝐞₃ + 1.66 𝐞₄,
  -1.25 𝐞₁ + 1.31 𝐞₂ + 2.28 𝐞₃ + 0.95 𝐞₄,
  0.102 𝐞₁ + 1.82 𝐞₂ + 0.847 𝐞₃ + 3.0 𝐞₄],
 [-0.916 𝐞₁ + 0.985 𝐞₂ + 2.27 𝐞₃ + 0.959 𝐞₄,
  -0.0284 𝐞₁ + 0.354 𝐞₂ + 2.41 𝐞₃ + 1.66 𝐞₄,
  -1.25 𝐞₁ + 1.31 𝐞₂ + 2.28 𝐞₃ + 0.95 𝐞₄,
  0.102 𝐞₁ + 1.82 𝐞₂ + 0.847 𝐞₃ + 3.0 𝐞₄])

In [29]:
#2.3 trace
def trace(f, alg):
    return sum(vr.sp(f(v)) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
def divergence(f, alg):
    return sum(vr | f(v) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
(
    derivative(f, e1, alg).grade(0),
    trace(f, alg),
    divergence(f, alg)
)

(-3.99, -3.99, -3.99)

In [30]:
#2.4
r = 2
h = 1e-2
F = lambda vectors: wedge([f(v) for v in vectors])
def char_multi_linear(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += wedge(r_vectors[::-1]) * wedge([f(v) for v in vectors])
    return sdf

(
    char_multi_linear(f, r, alg),
    simplicial_derivative(F, np.zeros(r), alg, h)
)

(0.549 + -3.42 𝐞₁₂ + -7.52 𝐞₁₃ + -9.59 𝐞₁₄ + -2.62 𝐞₂₃ + -9.4 𝐞₂₄ + -5.95 𝐞₃₄ + 2.45 𝐞₁₂₃₄,
 0.549 + -3.42 𝐞₁₂ + -7.52 𝐞₁₃ + -9.59 𝐞₁₄ + -2.62 𝐞₂₃ + -9.4 𝐞₂₄ + -5.95 𝐞₃₄ + 2.45 𝐞₁₂₃₄)

In [31]:
#2.6 when full rank, only scalar remains
def trans2matrix(f, alg):
    values = [f(a) for a in alg.frame]
    return np.array([[ar.sp(v)[0] for v in values] for ar in reciprocal(alg.frame)])
matrix = trans2matrix(f, alg)
I = alg.pseudoscalar((1,))
char_multi_linear(f, 4, alg)[0], np.linalg.det(matrix), det(f, I, I)

(3.09751097309765, 3.0975109730976462, 3.097510973097645)

In [32]:
#2.7 sum
r = 3
h = lambda x: f(x)+g(x)

# Sometimes I use a random frame to check frame independence
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
drF = 0
coef = [1/(factorial(r-s)*factorial(s)) for s in range(r+1)]
for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
    gh = 0
    for s in range(r+1):
        gh += coef[s]*wedge([f(v) for v in reci_vectors[:s]]) ^ wedge([g(v) for v in reci_vectors[s:]])
    drF += wedge(base_vectors[::-1]) * gh
char_multi_linear(h, r, alg), drF

(26.9 + 2.72 𝐞₁₂ + 27.8 𝐞₁₃ + 23.0 𝐞₁₄ + 16.1 𝐞₂₃ + 15.3 𝐞₂₄ + 23.6 𝐞₃₄,
 26.9 + 2.72 𝐞₁₂ + 27.8 𝐞₁₃ + 23.0 𝐞₁₄ + 16.1 𝐞₂₃ + 15.3 𝐞₂₄ + 23.6 𝐞₃₄)

In [33]:
#2.8 characteristic polynomial
def char_scalar(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += alg.sp(wedge(r_vectors[::-1]), wedge([f(v) for v in vectors]))[0]
    return sdf

# This calculates n characteristic multivectors. 
# But drops the nonscalar information
def char_poly_coefs(f, alg, frame=None, r_frame=None):
    n = alg.d
    if frame:
        n = len(frame)
    return [(-1)**(n-s) * char_scalar(f, s, alg, frame, r_frame) for s in range(n+1)]

def polynomial(l, coefs):
    return sum(l ** i * coef for i, coef in enumerate(coefs[::-1]))

l = 3.3
n = 4
(
    char_multi_linear(lambda x: f(x) - l*x, n, alg), 
    polynomial(l, char_poly_coefs(f, alg))
)

(2.45e+02, 245.37621303943115)

In [34]:
def r_fold(f,r,value):
    return reduce(lambda v, _: f(v), range(r), value)

r_fold(f, 3, x), f(f(f(x))), r_fold(f, 0, x), x

(0.274 𝐞₁ + 4.32 𝐞₂ + -1.38 𝐞₃ + 6.58 𝐞₄,
 0.274 𝐞₁ + 4.32 𝐞₂ + -1.38 𝐞₃ + 6.58 𝐞₄,
 0.817 𝐞₁ + 0.728 𝐞₂ + 0.501 𝐞₃ + 0.118 𝐞₄,
 0.817 𝐞₁ + 0.728 𝐞₂ + 0.501 𝐞₃ + 0.118 𝐞₄)

In [35]:
#2.9 Cayley Hamilton
sum(r_fold(f, i, x) * coef for i, coef in enumerate(char_poly_coefs(f, alg)[::-1]))

-3.55e-15 𝐞₁ + -1.24e-14 𝐞₂ + -1.07e-14 𝐞₃ + -1.73e-14 𝐞₄

In [36]:
#2.10 We can find eigenvalues like the usual many-fold method.
# How about eigenvectors? 
# What else can we get from characteristic multivector? Some papers I should go through...
matrix = trans2matrix(f, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
coefs = char_poly_coefs(f, alg)
eigvalues, np.roots(coefs)

(array([ 0.88745191+0.j        ,  0.47142062+0.j        ,
        -2.67628856+0.49128488j, -2.67628856-0.49128488j]),
 array([-2.67628856+0.49128488j, -2.67628856-0.49128488j,
         0.88745191+0.j        ,  0.47142062+0.j        ]))

In [37]:
# From Lasenby's paper, we can reconstruct a rotor from characteristic multivector:
# Note that the rotor must have non-zero scalar part
# the paper suggest such method is pretty robust for noisy data
R = blade_exp(create_r_blade(2, alg)) * blade_exp(create_r_blade(2, alg))
rotor = lambda x: R.sw(x) # a screw motion

terms_ratio(R.reverse(), sum(char_multi_linear(rotor, s, alg) for s in range(n+1)))

array([0.11015787, 0.11015787, 0.11015787, 0.11015787, 0.11015787,
       0.11015787, 0.11015787, 0.11015787])

In [38]:
# recovering matrix to transfromation
def matrix2trans(M, alg:Algebra ):
    d = alg.d
    assert M.shape[0] == d, 'dimension not fit'
    return lambda x: sum(c*e for c,e in zip(np.dot(M, x.asfullmv()[1:d+1]), alg.frame))
    
mf = matrix2trans(matrix, alg)
mf(x), f(x)

(0.0609 𝐞₁ + 1.77 𝐞₂ + 1.84 𝐞₃ + 3.15 𝐞₄,
 0.0609 𝐞₁ + 1.77 𝐞₂ + 1.84 𝐞₃ + 3.15 𝐞₄)

In [39]:
#2.11
s = 2
rhs = (1/s) * sum((-1)**(r+1) * char_scalar(f, s-r, alg) * trace(lambda x: r_fold(f, r, x), alg)[0] for r in range(1,s+1))
char_scalar(f, s, alg), rhs

(0.5487744027726744, 0.5487744027726773)

In [40]:
#2.13
k = 3
char_scalar(f, k, alg), sum(np.prod(comb) for comb in combinations(eigvalues, k))

(7.8216099935389565, (7.821609993538947+0j))

In [41]:
#2.14 It's easier to show the eigenvalues of f^k are {lambda^k_i} by linearity
trace(lambda x: r_fold(f, k, x), alg)[0], sum(l**k for l in eigvalues)

(-33.65853705557374, (-33.65853705557372+0j))

In [42]:
# 3.4 A space must bound f to own an invariant subspace.
I = alg.pseudoscalar((1,))
outermorphism(f, I, alg), det(f, I, I)*I, adjoint_outermorphism(f, I, alg)

(3.1 𝐞₁₂₃₄, 3.1 𝐞₁₂₃₄, 3.1 𝐞₁₂₃₄)

In [43]:
#3.6
l = 2.2
n = 4
F = lambda x: f(x) - l*x
coefs = char_poly_coefs(f, alg)
outermorphism(F, I, alg), polynomial(l, coefs) * I

(54.5 𝐞₁₂₃₄, 54.5 𝐞₁₂₃₄)

In [44]:
#3.7
I = e12
frame = [e1,e2]
g = lambda x: P((A*x.grade(1)*B).grade(1), I)
coefs = char_poly_coefs(g, alg, frame, frame)
F = lambda x: g(x) - l*x
outermorphism(F, I, alg), polynomial(l, coefs) * I

(11.6 𝐞₁₂, 11.6 𝐞₁₂)

In [45]:
#3.8
I = e123
frame2 = [e3]
frame3 = [e1,e2,e3]
h = lambda x: P((A*P(x, e12).grade(1)*B).grade(1), e12) + P((B*P(x, e3).grade(1)*C).grade(1), e3)
F = lambda x: h(x) - l*x
(
    char_scalar(F, 2, alg, frame, frame) * char_scalar(F, 1, alg, frame2, frame2),
    char_scalar(F, 3, alg, frame3, frame3)
)

(-38.297517413228206, -38.297517413228206)

In [46]:
outermorphism(h, e1, alg), adjoint_outermorphism(h, e1, alg)

(-1.88 𝐞₁ + 0.952 𝐞₂, -1.88 𝐞₁ + 0.504 𝐞₂)

In [47]:
#3.9a
adjoint_outermorphism(h, e123, alg) | e12, outermorphism(h, e12, alg).e12 * outermorphism(h, -e3, alg)

(1.05 𝐞₃, 1.05 𝐞₃)

In [48]:
#3.9b
outermorphism(h, e123, alg) | e3, adjoint_outermorphism(h, e3, alg).e3 * adjoint_outermorphism(h, e12, alg)

(-1.05 𝐞₁₂, -1.05 𝐞₁₂)

In [49]:
def separate_indices(numbers):
    real_indices = []
    conjugate_pairs_indices = []
    visited = set()  # To keep track of indices already processed

    for i, num in enumerate(numbers):
        if np.isreal(num):  # Check if the number is real
            real_indices.append(i)
        elif i not in visited:  # Check if the index is not already processed
            conjugate = np.conj(num)
            for j in range(i + 1, len(numbers)):
                if numbers[j] == conjugate:
                    conjugate_pairs_indices.append((i, j))
                    visited.add(i)
                    visited.add(j)
                    break

    return real_indices, conjugate_pairs_indices

separate_indices(eigvalues), eigvalues

(([0, 1], [(2, 3)]),
 array([ 0.88745191+0.j        ,  0.47142062+0.j        ,
        -2.67628856+0.49128488j, -2.67628856-0.49128488j]))

In [50]:
# An example of non proper blade:
def linalg_eigblades(f, alg: Algebra):
    matrix = trans2matrix(f, alg)
    eigvalues, eigvecs = np.linalg.eig(matrix)
    real_indices, conjugate_pairs_indices = separate_indices(eigvalues)
    eigblades = [alg.vector(np.real(eigvecs[:, i])) for i in real_indices]
    for k, l in conjugate_pairs_indices:
        v1 = np.real(eigvecs[:,k] + eigvecs[:,l])
        v2 = np.imag(eigvecs[:,k] - eigvecs[:,l])
        eigblades.append(alg.vector(v1) ^ alg.vector(v2))
    return eigblades 

def terms_ratio(A, B: MultiVector):
    valid_keys = [k for k in B.keys() if not np.isclose(B[k], 0)]
    return np.divide([A[k] for k in valid_keys], [B[k] for k in valid_keys])

eigblades = linalg_eigblades(f, alg)

# right eigenblades but not left!
for blade in eigblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )

[0.88745191 0.88745191 0.88745191 0.88745191] [-67.41721083   8.24157489   1.97535173  -0.14773723]
[0.47142062 0.47142062 0.47142062 0.47142062] [-18.88163855  16.53578149   1.79796503  -0.04534935]
[7.4038813 7.4038813 7.4038813 7.4038813 7.4038813 7.4038813] [ 3.63935719 10.70327492 14.22075579  5.06100541  7.22468922 36.89885214]


In [51]:
#3.11 reciprocal for blades
def reciprocal(blades):
    I = wedge(blades)
    dualblades = []
    for k in range(len(blades)):
        sign = (-1) ** (blades[k].grades[0] * sum(blade.grades[0] for blade in blades[:k]))
        dualblades.append(sign * wedge(blades[:k] + blades[k+1:]) * I.inv())
    return dualblades

dualblades = reciprocal(eigblades)
# The reciprocal of a right eigenblade is a left eigenblade! Note that they correspond to the same eigenvalues
for blade in dualblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )


[  0.37599112   2.58360865   3.66267446 -12.62557649] [0.88745191 0.88745191 0.88745191 0.88745191]
[ 0.4474008   2.5448294   4.36182348 -8.20656429] [0.47142062 0.47142062 0.47142062 0.47142062]
[31.36972628  4.62765037  8.66851587  7.9628419  10.99002913 -4.36325586] [7.4038813 7.4038813 7.4038813 7.4038813 7.4038813 7.4038813]


In [52]:
#3.12 
[eigblades[i] | dualblades[j] for i,j in [(0,0), (0,1), (1,1)]]

[1.0, -2.22e-16, 1.0]

In [53]:
# What transformation have a non-trivial proper blade?
# A invariant subspace, say e123 of e1234
# from 3.12 we know we're looking for blades equal to thier own reciprocal
# that means orthogonal to other eigenspaces

pf = lambda x: P(f(P(x, e123)), e123) + P(f(P(x, e4)), e4)
[(outermorphism(pf, blade, alg), adjoint_outermorphism(pf, blade, alg)) for blade in [e123, e4]]

[(8.28 𝐞₁₂₃, 8.28 𝐞₁₂₃), (-0.979 𝐞₄, -0.979 𝐞₄)]

In [54]:
eigblades = linalg_eigblades(pf, alg)
for E in [e123, e4]:
    for blade in eigblades: # not just vectors
        blade = (blade | E)*E
        om = outermorphism(pf, blade, alg)
        am = adjoint_outermorphism(pf, blade, alg)
        print(
            terms_ratio(om, blade), 
            terms_ratio(am, blade)
            )

[1.57806735 1.57806735 1.57806735] [4.34281059 1.38330886 0.55096219]
[-2.45058192 -2.45058192 -2.45058192] [-5.33653247 -2.6150299  -1.83237774]
[-2.14183758 -2.14183758 -2.14183758] [ 6.01753236 -3.25659777 -1.6633345 ]
[] []
[] []
[] []
[] []
[-0.97935244] [-0.97935244]


In [55]:
#4.1b A matrix from a symmetric transformation is symmetric
fp = lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2
trans2matrix(fp, alg), fp(x), derivative(lambda x: x | f(x)/2, x, alg, grade=1)

(array([[-1.17751062,  1.70594213,  0.98424924,  0.33906042],
        [ 1.70594213, -0.86390538,  1.31463271,  0.5226545 ],
        [ 0.98424924,  1.31463271, -0.97293615,  1.56945239],
        [ 0.33906042,  0.5226545 ,  1.56945239, -0.97935244]]),
 0.812 𝐞₁ + 1.49 𝐞₂ + 1.46 𝐞₃ + 1.33 𝐞₄,
 0.812 𝐞₁ + 1.49 𝐞₂ + 1.46 𝐞₃ + 1.33 𝐞₄)

In [56]:
# a curl is a part of a derivative
derivative(f, x, alg, grade=1).grade(2), curl(f, x, alg, grade=1)

(1.32 𝐞₁₂ + 0.47 𝐞₁₃ + 2.61 𝐞₁₄ + 0.679 𝐞₂₃ + 1.39 𝐞₂₄ + 1.01 𝐞₃₄,
 1.32 𝐞₁₂ + 0.47 𝐞₁₃ + 2.61 𝐞₁₄ + 0.679 𝐞₂₃ + 1.39 𝐞₂₄ + 1.01 𝐞₃₄)

In [57]:
#4.1c a skew transfromation leads to a skew matrix
fm = lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2
trans2matrix(fm, alg), fm(x), x | derivative(f, x, alg, grade=1).grade(2)/2

(array([[-1.11022302e-16, -6.59378854e-01, -2.35024838e-01,
         -1.30706450e+00],
        [ 6.59378854e-01,  5.55111512e-17, -3.39710365e-01,
         -6.95166724e-01],
        [ 2.35024838e-01,  3.39710365e-01, -3.33066907e-16,
         -5.04674365e-01],
        [ 1.30706450e+00,  6.95166724e-01,  5.04674365e-01,
         -2.22044605e-16]]),
 -0.751 𝐞₁ + 0.287 𝐞₂ + 0.38 𝐞₃ + 1.83 𝐞₄,
 -0.751 𝐞₁ + 0.287 𝐞₂ + 0.38 𝐞₃ + 1.83 𝐞₄)

In [58]:
#4.2 fp curl free
a | fp(x), x | fp(a), curl(fp, x, alg, grade=1)

(0.587,
 0.587,
 1.16e-10 𝐞₁₂ + 1.16e-10 𝐞₁₃ + -1.75e-10 𝐞₁₄ + -1.16e-10 𝐞₂₃ + 1.16e-10 𝐞₂₄ + 4.66e-10 𝐞₃₄)

In [59]:
#4.3 spectral decomposition of the symmetric part
matrix = trans2matrix(fp, alg)
eigvalues, eigvecs = np.linalg.eigh(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
mf(x), fp(x)

(0.812 𝐞₁ + 1.49 𝐞₂ + 1.46 𝐞₃ + 1.33 𝐞₄,
 0.812 𝐞₁ + 1.49 𝐞₂ + 1.46 𝐞₃ + 1.33 𝐞₄)

In [60]:
# QR factorization
transframe = [fp(e) for e in alg.frame] #another basis
gs_trans = gram_schmidt(transframe)
gs_trans2 = gram_schmidt([fp(v / norm(v)) for v in gs_trans[::-1]])[::-1]
gs_trans2


[-82.8 𝐞₁ + -2.49e-15 𝐞₂ + 1.23e-14 𝐞₃ + 1.67e-14 𝐞₄,
 4.84e-16 𝐞₁ + -16.1 𝐞₂ + -3.42e-15 𝐞₃ + -1.63e-15 𝐞₄,
 -4.42e-16 𝐞₁ + 6.31e-16 𝐞₂ + -2.97 𝐞₃ + -4.49e-16 𝐞₄,
 -2.22e-16 𝐞₁ + 1.11e-16 𝐞₂ + 1.67e-16 𝐞₃ + -1.1 𝐞₄]

In [61]:
# A GA version of eigenblade-solver given the eigenvalues
def null_space(f, alg, tol=1e-5, frame=None):
    if not frame:
        frame = alg.frame
    null_sp = wedge(frame)
    images = [f(a) for a in frame]
    for v in images:
        contracted = v | null_sp
        if normsq(contracted) >= tol:
            null_sp = contracted
    return null_sp

def eigenblades_from_values(f, eigvalues, alg, frame=None):
    eigblades = []
    for e in eigvalues:
        null_f = lambda x: f(x) - e*x
        eigblades.append(null_space(null_f, alg, frame=frame))
    return eigblades

In [62]:
# One thing to note for a symmetric transformation:
# the characteristic multivectors are of scalar values
# So all subspaces are invariant under the transfromation.
[char_multi_linear(fp, r, alg) for r in range(5)]

[1,
 -3.99 + -2.22e-16 𝐞₁₂ + -1.11e-16 𝐞₁₃ + 1.11e-16 𝐞₁₄ + 2.22e-16 𝐞₂₄,
 -2.5 + 1.33e-15 𝐞₁₂ + 4.44e-16 𝐞₁₃ + -6.66e-16 𝐞₁₄ + -4.44e-16 𝐞₂₃ + -4.44e-16 𝐞₂₄ + -8.88e-16 𝐞₃₄ + -1.67e-16 𝐞₁₂₃₄,
 21.3 + 1.11e-16 𝐞₁₂ + 8.88e-16 𝐞₁₃ + 8.88e-16 𝐞₂₄ + 1.11e-15 𝐞₃₄,
 -13.9]

In [63]:
np.roots(char_poly_coefs(fp, alg)), eigvalues

(array([ 2.29422941, -2.80333157, -2.68100398, -0.80359845]),
 array([-2.80333157, -2.68100398, -0.80359845,  2.29422941]))

In [64]:
eigblades = eigenblades_from_values(fp, eigvalues, alg)
[e/norm(e) for e in eigblades], eigvecs

([-0.403 𝐞₁ + 0.629 𝐞₂ + -0.552 𝐞₃ + 0.37 𝐞₄,
  -0.624 𝐞₁ + 0.338 𝐞₂ + 0.528 𝐞₃ + -0.466 𝐞₄,
  -0.478 𝐞₁ + -0.43 𝐞₂ + 0.324 𝐞₃ + 0.694 𝐞₄,
  -0.469 𝐞₁ + -0.553 𝐞₂ + -0.558 𝐞₃ + -0.404 𝐞₄],
 [-0.403 𝐞₁ + 0.629 𝐞₂ + -0.552 𝐞₃ + 0.37 𝐞₄,
  0.624 𝐞₁ + -0.338 𝐞₂ + -0.528 𝐞₃ + 0.466 𝐞₄,
  0.478 𝐞₁ + 0.43 𝐞₂ + -0.324 𝐞₃ + -0.694 𝐞₄,
  -0.469 𝐞₁ + -0.553 𝐞₂ + -0.558 𝐞₃ + -0.404 𝐞₄])

In [65]:
#4.4 modify mf so to have a repeat eigenvalue (degeneracy)
eigvalues[1] = eigvalues[0]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
for blade in [eigvecs[0], wedge(eigvecs[:2])]:
    of = outermorphism(mf, blade, alg)
    print(terms_ratio(of, blade))

[-2.80333157 -2.80333157 -2.80333157 -2.80333157]
[7.85866788 7.85866788 7.85866788 7.85866788 7.85866788 7.85866788]


In [66]:
eigblades = eigenblades_from_values(mf, set(eigvalues), alg)
[e/norm(e) for e in eigblades], [wedge(eigvecs[:2])] + eigvecs[2:]

([-0.478 𝐞₁ + -0.43 𝐞₂ + 0.324 𝐞₃ + 0.694 𝐞₄,
  -0.469 𝐞₁ + -0.553 𝐞₂ + -0.558 𝐞₃ + -0.404 𝐞₄,
  0.256 𝐞₁₂ + -0.557 𝐞₁₃ + 0.419 𝐞₁₄ + 0.519 𝐞₂₃ + -0.418 𝐞₂₄ + 0.0623 𝐞₃₄],
 [-0.256 𝐞₁₂ + 0.557 𝐞₁₃ + -0.419 𝐞₁₄ + -0.519 𝐞₂₃ + 0.418 𝐞₂₄ + -0.0623 𝐞₃₄,
  0.478 𝐞₁ + 0.43 𝐞₂ + -0.324 𝐞₃ + -0.694 𝐞₄,
  -0.469 𝐞₁ + -0.553 𝐞₂ + -0.558 𝐞₃ + -0.404 𝐞₄])

In [67]:
#4.6 a reflection is an orthogonal symmetric transformation 
A3 = create_r_blade(3, alg).normalized()
matrix = trans2matrix(lambda x: (-1)**3*(A3.sw(x)), alg)
# a symmetric matrix, orthogonal, unit eigvals
matrix, [np.dot(matrix[i], matrix[j]) for i,j in [(1,1), (1,2), (2,3), (3,3)]], np.linalg.eigvals(matrix)

(array([[-0.99664617,  0.07890486, -0.0141661 , -0.01642452],
        [ 0.07890486,  0.85637586, -0.33328244, -0.38641587],
        [-0.0141661 , -0.33328244, -0.9401645 ,  0.06937476],
        [-0.01642452, -0.38641587,  0.06937476, -0.9195652 ]]),
 [0.9999999999999994, -3.469446951953614e-18, 0.0, 0.9999999999999996],
 array([-1.,  1., -1., -1.]))

In [68]:
#4.6 Find blade A from an orthogonal-symmetric-transfromation?
ot = matrix2trans(matrix, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
A3_ = wedge([v for v, e in zip(eigvecs, eigvalues) if np.isclose(e, -1)])
(-1)**3*(A3.sw(x)), ot(x)

(-0.766 𝐞₁ + 0.475 𝐞₂ + -0.717 𝐞₃ + -0.368 𝐞₄,
 -0.766 𝐞₁ + 0.475 𝐞₂ + -0.717 𝐞₃ + -0.368 𝐞₄)

In [69]:
#4.7 Not true for whole space, but the proper ones
# In matrix terms: diagonal parts commute
eigblades = linalg_eigblades(pf, alg)
eigblades.append(e123)
eigblades, [max_diff(pf(blade.sw(x)), blade.sw(pf(x))) for blade in eigblades]

([-0.42 𝐞₁ + -0.657 𝐞₂ + -0.627 𝐞₃,
  -0.234 𝐞₁ + 0.738 𝐞₂ + -0.633 𝐞₃,
  -0.042 𝐞₁ + -0.556 𝐞₂ + 0.83 𝐞₃,
  1.0 𝐞₄,
  1 𝐞₁₂₃],
 [0.6996330203352243, 0.3949230021863379, 0.6143734393231663, 0.0, 0.0])

In [70]:
#4.8 skew transformation -> skew bilinear form -> bivector
fm_biform = lambda vectors: vectors[0] | fm(vectors[1])
F = simplicial_derivative(fm_biform, np.zeros(2), alg)
a | fm(x), a| (x | F)

(0.287, 0.287)

In [71]:
#4.9 so fm(x) is always orthogonal to x
fm(x), x|F, fm(x) | x

(-0.751 𝐞₁ + 0.287 𝐞₂ + 0.38 𝐞₃ + 1.83 𝐞₄,
 -0.751 𝐞₁ + 0.287 𝐞₂ + 0.38 𝐞₃ + 1.83 𝐞₄,
 2.78e-17)

In [72]:
#4.10 F is a bivector
derivative(fm, x, alg, grade=1), curl(fm, x, alg, grade=1), curl(f, x, alg, grade=1), 2*F

(1.16e-10 + 1.32 𝐞₁₂ + 0.47 𝐞₁₃ + 2.61 𝐞₁₄ + 0.679 𝐞₂₃ + 1.39 𝐞₂₄ + 1.01 𝐞₃₄,
 1.32 𝐞₁₂ + 0.47 𝐞₁₃ + 2.61 𝐞₁₄ + 0.679 𝐞₂₃ + 1.39 𝐞₂₄ + 1.01 𝐞₃₄,
 1.32 𝐞₁₂ + 0.47 𝐞₁₃ + 2.61 𝐞₁₄ + 0.679 𝐞₂₃ + 1.39 𝐞₂₄ + 1.01 𝐞₃₄,
 1.32 𝐞₁₂ + 0.47 𝐞₁₃ + 2.61 𝐞₁₄ + 0.679 𝐞₂₃ + 1.39 𝐞₂₄ + 1.01 𝐞₃₄)

In [73]:
#4.11 bivector split commutes, you can think about a screw motion splits into translation and rotation
e13*e24, e24*e13

(-1 𝐞₁₂₃₄, -1 𝐞₁₂₃₄)

In [74]:
#4.13 How to factor F?
def wedge_power(A, r):
    if r == 0:
        return 1
    result = A
    for _ in range(r-1):
        result = A ^ result
    return result

Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(3)]
Ck, [char_multi_linear(fm, r, alg) for r in range(1, 5)] # these are full story, but we just take a fraction

([1.0,
  0.659 𝐞₁₂ + 0.235 𝐞₁₃ + 1.31 𝐞₁₄ + 0.34 𝐞₂₃ + 0.695 𝐞₂₄ + 0.505 𝐞₃₄,
  0.613 𝐞₁₂₃₄],
 [-6.11e-16 + 1.32 𝐞₁₂ + 0.47 𝐞₁₃ + 2.61 𝐞₁₄ + 0.679 𝐞₂₃ + 1.39 𝐞₂₄ + 1.01 𝐞₃₄,
  3.05 + -6.66e-16 𝐞₁₂ + -6.66e-16 𝐞₁₃ + -1.01e-15 𝐞₁₄ + -3.33e-16 𝐞₂₃ + -1.19e-15 𝐞₂₄ + -3.33e-16 𝐞₃₄ + 2.45 𝐞₁₂₃₄,
  -1.5e-15 + 0.619 𝐞₁₂ + -0.853 𝐞₁₃ + 0.417 𝐞₁₄ + 1.6 𝐞₂₃ + -0.288 𝐞₂₄ + 0.809 𝐞₃₄,
  0.376])

In [75]:
[alg.ip(Ck[i-1], Ck[i]) for i in range(1, len(Ck))]

[0.659 𝐞₁₂ + 0.235 𝐞₁₃ + 1.31 𝐞₁₄ + 0.34 𝐞₂₃ + 0.695 𝐞₂₄ + 0.505 𝐞₃₄,
 -0.31 𝐞₁₂ + 0.426 𝐞₁₃ + -0.208 𝐞₁₄ + -0.802 𝐞₂₃ + 0.144 𝐞₂₄ + -0.404 𝐞₃₄]

In [76]:
set.union(*(set(ck.keys()) for ck in Ck if isinstance(ck, MultiVector)))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [77]:
x|F, fm(x), x|F|F, fm(fm(x))

(-0.751 𝐞₁ + 0.287 𝐞₂ + 0.38 𝐞₃ + 1.83 𝐞₄,
 -0.751 𝐞₁ + 0.287 𝐞₂ + 0.38 𝐞₃ + 1.83 𝐞₄,
 -2.67 𝐞₁ + -1.89 𝐞₂ + -1.0 𝐞₃ + -0.591 𝐞₄,
 -2.67 𝐞₁ + -1.89 𝐞₂ + -1.0 𝐞₃ + -0.591 𝐞₄)

In [78]:
#4.14 4.18 Why square of eigen values? 
# The odd grade characteristic multivectors have zero scalar
# A correction on Lasenby's paper, not square of f, but the even outermorphism f_(x, y)
Ck2 = [(-1)**(2 - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(3)]
char_poly = char_poly_coefs(fm, alg)
Lk = np.roots(Ck2)

Ck2, char_poly, Lk, np.sqrt(-np.array(Lk)), np.roots(char_poly), char_poly_coefs(lambda x: fm(fm(x)), alg)

([1.0, 3.051790886738669, 0.3762761425668448],
 [1,
  6.106226635438361e-16,
  3.05179088673867,
  1.4988010832439613e-15,
  0.37627614256684516],
 array([-2.92306428, -0.12872661]),
 array([1.70969713, 0.3587849 ]),
 array([-5.11907019e-17+1.70969713j, -5.11907019e-17-1.70969713j,
        -2.54120630e-16+0.3587849j , -2.54120630e-16-0.3587849j ]),
 [1,
  6.103581773477341,
  10.065979901514892,
  2.2966322055653583,
  0.14158373546498468])

In [79]:
Ck[1], Ck[1] | Ck[2]

(0.659 𝐞₁₂ + 0.235 𝐞₁₃ + 1.31 𝐞₁₄ + 0.34 𝐞₂₃ + 0.695 𝐞₂₄ + 0.505 𝐞₃₄,
 -0.31 𝐞₁₂ + 0.426 𝐞₁₃ + -0.208 𝐞₁₄ + -0.802 𝐞₂₃ + 0.144 𝐞₂₄ + -0.404 𝐞₃₄)

In [80]:
#4.15
mv_map1 = {}
mv_map2 = {}
ck1, ck2 = Ck[1], Ck[1] | Ck[2]
for key in ck2.keys():
    if np.isclose(ck1[key],0) and np.isclose(ck2[key],0):
        continue
    ans = np.linalg.solve(np.array([[1, 1], Lk]), [ck1[key], ck2[key]])
    mv_map1[key], mv_map2[key] = ans
F1 = alg.multivector(mv_map1)
F2 = alg.multivector(mv_map2)
F1 + F2, F, F1*F2, F2*F1, F**2, F1**2, F2**2


(0.659 𝐞₁₂ + 0.235 𝐞₁₃ + 1.31 𝐞₁₄ + 0.34 𝐞₂₃ + 0.695 𝐞₂₄ + 0.505 𝐞₃₄,
 0.659 𝐞₁₂ + 0.235 𝐞₁₃ + 1.31 𝐞₁₄ + 0.34 𝐞₂₃ + 0.695 𝐞₂₄ + 0.505 𝐞₃₄,
 -6.25e-17 + 2.78e-17 𝐞₁₂ + 3.47e-18 𝐞₂₄ + -7.81e-18 𝐞₃₄ + 0.613 𝐞₁₂₃₄,
 -6.25e-17 + -2.6e-17 𝐞₁₂ + 8.67e-19 𝐞₁₃ + -6.94e-18 𝐞₂₃ + 0.613 𝐞₁₂₃₄,
 -3.05 + 4.16e-17 𝐞₁₄ + 1.23 𝐞₁₂₃₄,
 -0.129 + -6.51e-19 𝐞₁₂ + 2.17e-19 𝐞₁₃ + 1.73e-17 𝐞₁₂₃₄,
 -2.92 + -5.55e-17 𝐞₁₄ + 1.94e-16 𝐞₁₂₃₄)

In [81]:
#4.16
def normsq(A):
    return A.reverse().sp(A)[0]
(
    [(-normsq(F) - np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2,
    (-normsq(F) + np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2],
    Lk
)


([-2.9230642812441467, -0.12872660549452242],
 array([-2.92306428, -0.12872661]))

In [82]:
#4.17
F2_ = F / (1 + (1/2/Lk[0])*(F ^ F))
F1_ = F / (1 + (1/2/Lk[1])*(F ^ F))
F1_, F1, F2_, F2

(0.0804 𝐞₁₂ + -0.163 𝐞₁₃ + 0.0144 𝐞₁₄ + 0.271 𝐞₂₃ + -0.0836 𝐞₂₄ + 0.121 𝐞₃₄,
 0.0804 𝐞₁₂ + -0.163 𝐞₁₃ + 0.0144 𝐞₁₄ + 0.271 𝐞₂₃ + -0.0836 𝐞₂₄ + 0.121 𝐞₃₄,
 0.579 𝐞₁₂ + 0.398 𝐞₁₃ + 1.29 𝐞₁₄ + 0.0684 𝐞₂₃ + 0.779 𝐞₂₄ + 0.383 𝐞₃₄,
 0.579 𝐞₁₂ + 0.398 𝐞₁₃ + 1.29 𝐞₁₄ + 0.0684 𝐞₂₃ + 0.779 𝐞₂₄ + 0.383 𝐞₃₄)

In [83]:
def bivector_split(F, alg: Algebra):
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(m+1)]
    Ck2 = [(-1)**(2 - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    mv_map_list = [{} for _ in range(m)]
    ck_inner_list = [alg.ip(Ck[i-1], Ck[i]) for i in range(1, len(Ck))]
    mv_keys = set.union(*(set(ck.keys()) for ck in ck_inner_list if isinstance(ck, MultiVector)))
    inv_matrix = np.linalg.inv(np.array([np.ones(m), Lk]))
    for key in mv_keys:
        if all([np.isclose(ck[key],0) for ck in ck_inner_list]):
            continue
        ans = np.dot(inv_matrix, [ck[key] for ck in ck_inner_list])
        for i, v in enumerate(ans):
            mv_map_list[i][key] = v
    return [alg.multivector(mv_map) for mv_map in mv_map_list]

In [84]:
# What if we use the above function to split a bivector that's almost a blade?
# The result blades are R itself and its dual with a small magnitude
# Should assert not simple before process?

R = create_r_blade(2, alg) + 1e-12 * A # a blade with a small perturbation
blades = bivector_split(R, alg)
R**2, [terms_ratio(b, r) for r in [R, R.dual()] for b in blades]

(-1.72 + -7.94e-13 𝐞₁ + -6e-13 𝐞₂ + 7.09e-13 𝐞₃ + 1.12e-12 𝐞₄ + -1.48e-12 𝐞₁₂ + 4.17e-13 𝐞₁₃ + 1.03e-13 𝐞₁₄ + -1.22e-13 𝐞₂₃ + -8.31e-13 𝐞₂₄ + 1.69e-12 𝐞₃₄ + -6.13e-13 𝐞₁₂₃ + -1.21e-12 𝐞₁₂₄ + -1.21e-12 𝐞₁₃₄ + -5.11e-13 𝐞₂₃₄ + -3.92e-13 𝐞₁₂₃₄,
 [array([ 8.07258580e-14, -8.75576655e-14,  7.31141275e-14,  1.77494263e-13,
         -1.48214758e-13,  1.60758133e-13]),
  array([1., 1., 1., 1., 1., 1.]),
  array([1.13918121e-13, 1.13918121e-13, 1.13918121e-13, 1.13918121e-13,
         1.13918121e-13, 1.13918121e-13]),
  array([ 1.41117262, -1.30106393,  1.5580863 ,  0.64181297, -0.76860174,
          0.70863053])])

In [85]:
# double skew is symmetric
fm2 = lambda x: fm(fm(x))
curl(fm2, x, alg, h=1e-2, grade=1)

2.84e-14 𝐞₁₂ + -6.75e-14 𝐞₁₄ + 7.11e-14 𝐞₂₄ + 3.55e-14 𝐞₃₄

In [86]:
#4.19
v = P(e1, F1)
fm2(v), (F1 ** 2) * v

(-0.0334 𝐞₁ + 0.0455 𝐞₂ + 0.0201 𝐞₃ + -0.0266 𝐞₄,
 -0.0334 𝐞₁ + 0.0455 𝐞₂ + 0.0201 𝐞₃ + -0.0266 𝐞₄ + 3.76e-18 𝐞₁₂₃ + 2.57e-18 𝐞₁₂₄ + -6.09e-18 𝐞₁₃₄ + -4.5e-18 𝐞₂₃₄)

In [87]:
#4.20 
F1_eigvecs= [v.normalized() for v in gram_schmidt(blade_split(F1, alg))]
F1, np.sqrt(normsq(F1)) * wedge(F1_eigvecs)


(0.0804 𝐞₁₂ + -0.163 𝐞₁₃ + 0.0144 𝐞₁₄ + 0.271 𝐞₂₃ + -0.0836 𝐞₂₄ + 0.121 𝐞₃₄,
 0.0804 𝐞₁₂ + -0.163 𝐞₁₃ + 0.0144 𝐞₁₄ + 0.271 𝐞₂₃ + -0.0836 𝐞₂₄ + 0.121 𝐞₃₄)

In [88]:
# If Lk[0] == Lk[1], the linear equation has no unique solution in 4.15
# nonunique bivector split? We should find another split of e12 + e34. 
t = 1.1
R = blade_exp(t*e13) * blade_exp(t*e24)
a,b,c,d = [R.sw(v) for v in alg.frame]
a,b,c,d, a*b+c*d, a*b, c*d

(-0.589 𝐞₁ + -0.808 𝐞₃,
 -0.589 𝐞₂ + -0.808 𝐞₄,
 0.808 𝐞₁ + -0.589 𝐞₃,
 0.808 𝐞₂ + -0.589 𝐞₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 𝐞₁₂ + 0.476 𝐞₁₄ + -0.476 𝐞₂₃ + 0.654 𝐞₃₄,
 0.654 𝐞₁₂ + -0.476 𝐞₁₄ + 0.476 𝐞₂₃ + 0.346 𝐞₃₄)

In [89]:
R.sw(e12 + e34), (e12+e34) * (R**2), R**2

(1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 + -0.476 𝐞₁₃ + -0.476 𝐞₂₄ + -0.654 𝐞₁₂₃₄)

In [90]:
#4.21
B2 = B.grade(2)
# fm = lambda x: f(x) - adjoint_outermorphism(f, x, alg)
# F = curl(fm, x, alg, grade=1) / 2
outermorphism(fm, B2, alg), (0.5 * (B2 | (F^F))) - ((B2|F) * F)

(0.685 𝐞₁₂ + 0.693 𝐞₁₃ + 1.48 𝐞₁₄ + 0.249 𝐞₂₃ + 1.18 𝐞₂₄ + 0.731 𝐞₃₄,
 0.685 𝐞₁₂ + 0.693 𝐞₁₃ + 1.48 𝐞₁₄ + 0.249 𝐞₂₃ + 1.18 𝐞₂₄ + 0.731 𝐞₃₄)

In [91]:
# What is a symmetric bivector function?
# I guess it means:
outermorphism(fm, B2, alg), adjoint_outermorphism(fm, B2, alg)

(0.685 𝐞₁₂ + 0.693 𝐞₁₃ + 1.48 𝐞₁₄ + 0.249 𝐞₂₃ + 1.18 𝐞₂₄ + 0.731 𝐞₃₄,
 0.685 𝐞₁₂ + 0.693 𝐞₁₃ + 1.48 𝐞₁₄ + 0.249 𝐞₂₃ + 1.18 𝐞₂₄ + 0.731 𝐞₃₄)

In [92]:
#4.23 The bivector split solves the eigenbivector problem of a skew transformation
outermorphism(fm, F1, alg), normsq(F1)*F1

(0.0104 𝐞₁₂ + -0.021 𝐞₁₃ + 0.00185 𝐞₁₄ + 0.0349 𝐞₂₃ + -0.0108 𝐞₂₄ + 0.0156 𝐞₃₄,
 0.0104 𝐞₁₂ + -0.021 𝐞₁₃ + 0.00185 𝐞₁₄ + 0.0349 𝐞₂₃ + -0.0108 𝐞₂₄ + 0.0156 𝐞₃₄)

In [93]:
#5.1 normal transformation
af = lambda x: adjoint_outermorphism(f, x, alg)

fp(fm(x)), fm(fp(x)), af(f(x)), f(af(x)) # not normal

(2.37 𝐞₁ + -0.0757 𝐞₂ + 2.13 𝐞₃ + -1.3 𝐞₄,
 -3.06 𝐞₁ + -0.883 𝐞₂ + 0.0251 𝐞₃ + 2.83 𝐞₄,
 11.6 𝐞₁ + 5.42 𝐞₂ + 6.53 𝐞₃ + -1.5 𝐞₄,
 0.703 𝐞₁ + 3.8 𝐞₂ + 2.31 𝐞₃ + 6.76 𝐞₄)

In [94]:
M = blade_exp(create_r_blade(2, alg))
Mdual = blade_split(M.grade(2).dual(), alg)
rigid = lambda x: 1.1 *  M.sw(x) - 4.4 * P(x, Mdual[0]) + 3.3 * P(x, Mdual[1])
adrigid = lambda x: adjoint_outermorphism(rigid, x, alg)
adrigid(rigid(x)), rigid(adrigid(x))

(2.8 𝐞₁ + 9.87 𝐞₂ + -2.84 𝐞₃ + -1.83 𝐞₄,
 2.8 𝐞₁ + 9.87 𝐞₂ + -2.84 𝐞₃ + -1.83 𝐞₄)

In [95]:
rigid(x), 3.3 * M**2 * P(x, M.grade(2).normalized()) + 3.3 * P(x, M.grade(2).normalized().dual()) - 4.4 * P(x, Mdual[0]) + 5.5 * P(x, Mdual[1])

(-1.06 𝐞₁ + 2.24 𝐞₂ + 0.329 𝐞₃ + -1.26 𝐞₄,
 -0.923 𝐞₁ + 4.17 𝐞₂ + 0.072 𝐞₃ + -3.91 𝐞₄ + 5.84e-17 𝐞₁₂₃ + -8.86e-18 𝐞₁₂₄ + -2.19e-16 𝐞₁₃₄ + -6.82e-18 𝐞₂₃₄)

In [96]:
#5.2
def norm(A):
    return np.sqrt(normsq(A))
b1, b2 = norm(F1), norm(F2)
i1, i2 = F1/b1, F2/b2

F, b1*i1 + b2*i2, outermorphism(fm, i1, alg), b1**2*i1

(0.659 𝐞₁₂ + 0.235 𝐞₁₃ + 1.31 𝐞₁₄ + 0.34 𝐞₂₃ + 0.695 𝐞₂₄ + 0.505 𝐞₃₄,
 0.659 𝐞₁₂ + 0.235 𝐞₁₃ + 1.31 𝐞₁₄ + 0.34 𝐞₂₃ + 0.695 𝐞₂₄ + 0.505 𝐞₃₄,
 0.0289 𝐞₁₂ + -0.0586 𝐞₁₃ + 0.00515 𝐞₁₄ + 0.0973 𝐞₂₃ + -0.03 𝐞₂₄ + 0.0436 𝐞₃₄,
 0.0289 𝐞₁₂ + -0.0586 𝐞₁₃ + 0.00515 𝐞₁₄ + 0.0973 𝐞₂₃ + -0.03 𝐞₂₄ + 0.0436 𝐞₃₄)

In [97]:
# 5.3
terms_ratio(outermorphism(fp, i1, alg), i1) # not normal

array([13.75930442,  6.60739507, 58.14385927,  3.70455092, 13.80355869,
        3.02945907])

In [98]:
# 5.3
def sym_part(f, alg):
    return lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2

def skew_part(f, alg):
    return lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2

R = curl(rigid, x, alg, grade=1)/2
skew_blades = bivector_split(R, alg)
skew_eigs = [norm(b) for b in skew_blades]
skew_blades = [b / e for b,e in zip(skew_blades, skew_eigs) if e > 1e-6]
skew_eigs = [e for e in skew_eigs if e > 1e-6]
rigid_p = sym_part(rigid, alg)
[terms_ratio(outermorphism(rigid_p, i, alg), i) for i in skew_blades]

[array([0.02636475, 0.02636475, 0.02636475, 0.02636475, 0.02636475,
        0.02636475])]

In [99]:
#5.4
rand_vec = create_r_vectors(1, alg)[0]
sym_vecs = [(rand_vec | i).normalized() for i in skew_blades]
other_sym_space = wedge(skew_blades).dual()
sym_eigs = [terms_ratio(rigid_p(a), a)[0] for a in sym_vecs]

sym_eigs, np.square(sym_eigs), [terms_ratio(outermorphism(rigid_p, i, alg), i)[0] for i in skew_blades]


([-0.1623722571231166], array([0.02636475]), [0.026364749203253858])

In [100]:
#5.5a
m = len(skew_blades)
rigid_eigs = [sym_eigs[k] + skew_blades[k] * skew_eigs[k] for k in range(m)]
[rigid(a) for a in sym_vecs], [a*l for a, l in zip(sym_vecs,rigid_eigs)], [l.reverse()*a for a, l in zip(sym_vecs,rigid_eigs)]

([-0.569 𝐞₁ + -0.259 𝐞₂ + -0.572 𝐞₃ + -0.701 𝐞₄],
 [-0.569 𝐞₁ + -0.259 𝐞₂ + -0.572 𝐞₃ + -0.701 𝐞₄ + 3.47e-18 𝐞₁₂₄ + -5.55e-17 𝐞₁₃₄],
 [-0.569 𝐞₁ + -0.259 𝐞₂ + -0.572 𝐞₃ + -0.701 𝐞₄ + 5.55e-17 𝐞₁₃₄ + 6.94e-18 𝐞₂₃₄])

In [101]:
#5.5b
[adjoint_outermorphism(rigid, a, alg) for a in sym_vecs], [l*a for a, l in zip(sym_vecs,rigid_eigs)], [a*l.reverse() for a, l in zip(sym_vecs,rigid_eigs)]

([0.464 𝐞₁ + 0.237 𝐞₂ + 0.344 𝐞₃ + 0.906 𝐞₄],
 [0.464 𝐞₁ + 0.237 𝐞₂ + 0.344 𝐞₃ + 0.906 𝐞₄ + -5.55e-17 𝐞₁₃₄ + -6.94e-18 𝐞₂₃₄],
 [0.464 𝐞₁ + 0.237 𝐞₂ + 0.344 𝐞₃ + 0.906 𝐞₄ + -3.47e-18 𝐞₁₂₄ + 5.55e-17 𝐞₁₃₄])

In [102]:
[norm(l) for l in rigid_eigs], [np.sqrt(sym_eigs[k] ** 2 + skew_eigs[k] ** 2) for k in range(m)]

([1.1000000000729744], [1.1000000000729744])

In [103]:
#5.6
rho = [norm(l) for l in rigid_eigs] # dilation
theta = [np.arccos(sym_eigs[k] / rho[k]) for k in range(m)] # rotation angle

rigid_eigs, [rho[k] * blade_exp(theta[k] * skew_blades[k]) for k in range(m)] # skew_blades gives rotation plane

([-0.162 + -0.0459 𝐞₁₂ + 0.215 𝐞₁₃ + -0.586 𝐞₁₄ + 0.144 𝐞₂₃ + -0.21 𝐞₂₄ + -0.853 𝐞₃₄],
 [-0.162 + -0.0459 𝐞₁₂ + 0.215 𝐞₁₃ + -0.586 𝐞₁₄ + 0.144 𝐞₂₃ + -0.21 𝐞₂₄ + -0.853 𝐞₃₄])

In [104]:
np.sqrt(1 - (skew_eigs[0]/rho[0])**2) * rho[0]

0.16237225712311654

In [105]:
# spectral decomposition of symmetric part in other_sym_space
other_sym_frame = blade_split(other_sym_space, alg)
other_sym_eigs = np.roots(char_poly_coefs(rigid_p, alg, other_sym_frame, reciprocal(other_sym_frame)))
other_sym_eigs

array([ 4.34942367, -3.24942367])

In [106]:
#5.9 spectral decomposition of a normal transformation
sym_vecs_dual = [a|i for a, i in zip(sym_vecs, skew_blades)]
other_sym_blades = eigenblades_from_values(rigid_p, other_sym_eigs, alg, other_sym_frame)

rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs + sym_vecs_dual + other_sym_blades, rigid_eigs * 2 + list(other_sym_eigs))])

(-1.06 𝐞₁ + 2.24 𝐞₂ + 0.329 𝐞₃ + -1.26 𝐞₄,
 -1.06 𝐞₁ + 2.24 𝐞₂ + 0.329 𝐞₃ + -1.26 𝐞₄ + 2.43e-17 𝐞₁₂₃ + -1.04e-17 𝐞₁₂₄ + -8.33e-17 𝐞₁₃₄ + -1.39e-17 𝐞₂₃₄)

In [107]:
#5.17
sym_eigs, sym_vecs = np.linalg.eigh(trans2matrix(rigid_p, alg))
sym_vecs = [alg.vector(v) for v in sym_vecs.transpose()]
sym_eigs, sym_vecs

(array([-3.24942367, -0.16237226, -0.16237226,  4.34942367]),
 [-0.816 𝐞₁ + 0.091 𝐞₂ + 0.538 𝐞₃ + 0.19 𝐞₄,
  0.563 𝐞₁ + 0.236 𝐞₂ + 0.66 𝐞₃ + 0.438 𝐞₄,
  0.116 𝐞₁ + -0.0264 𝐞₂ + 0.486 𝐞₃ + -0.866 𝐞₄,
  0.0574 𝐞₁ + -0.967 𝐞₂ + 0.198 𝐞₃ + 0.148 𝐞₄])

In [108]:
# In my case, all vectors are both eigenvectors of rigid_p and rigid_n
rigid_n = lambda x: rigid(adjoint_outermorphism(rigid, x, alg))
[terms_ratio(rigid_n(v), v) for v in sym_vecs]

[array([10.55875417, 10.55875417, 10.55875417, 10.55875417]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([18.91748623, 18.91748623, 18.91748623, 18.91748623])]

In [109]:
# the same spectral decomposition of the normal transformation
beta = [v ^ rigid(v) for v in sym_vecs]
rigid_eigs = [a + b for a,b in zip(sym_eigs, beta)]
rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs, rigid_eigs)])

(-1.06 𝐞₁ + 2.24 𝐞₂ + 0.329 𝐞₃ + -1.26 𝐞₄,
 -1.06 𝐞₁ + 2.24 𝐞₂ + 0.329 𝐞₃ + -1.26 𝐞₄ + 6.58e-18 𝐞₁₂₃ + 5e-18 𝐞₁₂₄ + 5.65e-17 𝐞₁₃₄ + 2.42e-17 𝐞₂₃₄)

In [110]:
#5.19 our function "rigid" is normal but not orthogonal
A, outermorphism(rigid, adjoint_outermorphism(rigid, A, alg, h=1e-2), alg, h=1e-2)

(0.371 + 0.0529 𝐞₁ + 0.148 𝐞₂ + 0.544 𝐞₃ + 0.825 𝐞₄ + 0.854 𝐞₁₂ + 0.321 𝐞₁₃ + 0.52 𝐞₁₄ + 0.871 𝐞₂₃ + 0.173 𝐞₂₄ + 0.831 𝐞₃₄ + 0.253 𝐞₁₂₃ + 0.0885 𝐞₁₂₄ + 0.663 𝐞₁₃₄ + 0.223 𝐞₂₃₄ + 0.854 𝐞₁₂₃₄,
 0.371 + -3.05 𝐞₁ + -1.01 𝐞₂ + 3.08 𝐞₃ + 1.98 𝐞₄ + 1.66e+02 𝐞₁₂ + -32.7 𝐞₁₃ + -24.5 𝐞₁₄ + 1.24e+02 𝐞₂₃ + 37.9 𝐞₂₄ + 12.8 𝐞₃₄ + 72.6 𝐞₁₂₃ + 20.9 𝐞₁₂₄ + 16.8 𝐞₁₃₄ + 3.3 𝐞₂₃₄ + 2.5e+02 𝐞₁₂₃₄)

In [111]:
#5.19 But if we normalize each "eigenvalues" in the spectral decomposition
ortho_eigs = [l.normalized() for l in rigid_eigs]
ortho = lambda x: sum([P(x, a) * l for a, l in zip(sym_vecs, ortho_eigs)])
A, outermorphism(ortho, adjoint_outermorphism(ortho, A, alg, h=1e-2), alg, h=1e-2)

(0.371 + 0.0529 𝐞₁ + 0.148 𝐞₂ + 0.544 𝐞₃ + 0.825 𝐞₄ + 0.854 𝐞₁₂ + 0.321 𝐞₁₃ + 0.52 𝐞₁₄ + 0.871 𝐞₂₃ + 0.173 𝐞₂₄ + 0.831 𝐞₃₄ + 0.253 𝐞₁₂₃ + 0.0885 𝐞₁₂₄ + 0.663 𝐞₁₃₄ + 0.223 𝐞₂₃₄ + 0.854 𝐞₁₂₃₄,
 0.371 + 0.0529 𝐞₁ + 0.148 𝐞₂ + 0.544 𝐞₃ + 0.825 𝐞₄ + 0.854 𝐞₁₂ + 0.321 𝐞₁₃ + 0.52 𝐞₁₄ + 0.871 𝐞₂₃ + 0.173 𝐞₂₄ + 0.831 𝐞₃₄ + 0.253 𝐞₁₂₃ + 0.0885 𝐞₁₂₄ + 0.663 𝐞₁₃₄ + 0.223 𝐞₂₃₄ + 0.854 𝐞₁₂₃₄)

In [112]:
#5.20 left/right product of exponential
ortho(x), sum([l.inv() * P(x, a) for a, l in zip(sym_vecs, ortho_eigs)])

(-0.376 𝐞₁ + 0.439 𝐞₂ + 0.29 𝐞₃ + -1.02 𝐞₄ + 5.49e-18 𝐞₁₂₃ + 5.64e-18 𝐞₁₂₄ + -7.18e-17 𝐞₁₃₄ + -1.14e-17 𝐞₂₃₄,
 -0.376 𝐞₁ + 0.439 𝐞₂ + 0.29 𝐞₃ + -1.02 𝐞₄ + -1.1e-17 𝐞₁₂₃ + -1.91e-17 𝐞₁₂₄ + -1.5e-17 𝐞₁₃₄ + -7.74e-18 𝐞₂₃₄)

In [113]:
#5.21
(
    [sym_vecs[k-1].normalized() * sym_vecs[k].normalized() for k in range(1, 4)],
    [e.grade(2).normalized() for e in ortho_eigs]
)


([-1.94e-16 + -0.244 𝐞₁₂ + -0.841 𝐞₁₃ + -0.465 𝐞₁₄ + -0.0669 𝐞₂₃ + -0.00493 𝐞₂₄ + 0.11 𝐞₃₄,
  -1.11e-16 + -0.0422 𝐞₁₂ + 0.197 𝐞₁₃ + -0.538 𝐞₁₄ + 0.132 𝐞₂₃ + -0.193 𝐞₂₄ + -0.784 𝐞₃₄,
  5.55e-17 + -0.11 𝐞₁₂ + -0.00493 𝐞₁₃ + 0.0669 𝐞₁₄ + 0.465 𝐞₂₃ + -0.841 𝐞₂₄ + 0.244 𝐞₃₄],
 [0.647 𝐞₁₂ + 0.619 𝐞₁₃ + -0.03 𝐞₁₄ + 0.301 𝐞₂₃ + 0.229 𝐞₂₄ + 0.233 𝐞₃₄,
  -0.0422 𝐞₁₂ + 0.197 𝐞₁₃ + -0.538 𝐞₁₄ + 0.132 𝐞₂₃ + -0.193 𝐞₂₄ + -0.784 𝐞₃₄,
  -0.0422 𝐞₁₂ + 0.197 𝐞₁₃ + -0.538 𝐞₁₄ + 0.132 𝐞₂₃ + -0.193 𝐞₂₄ + -0.784 𝐞₃₄,
  -0.363 𝐞₁₂ + 0.105 𝐞₁₃ + 0.0271 𝐞₁₄ + -0.867 𝐞₂₃ + 0.288 𝐞₂₄ + -0.148 𝐞₃₄])

In [114]:
#5.23
def simple_rotor_sqrt(R):
    # for rotations only, don't use on screw motions
    R_norm = norm(R+1)
    assert R_norm >= 1e-4, "no explicit square root for -1"
    return (R + 1)/R_norm
    
neg_eig_pairs = []
eig_rotors = []
for i, e in enumerate(ortho_eigs):
    if normsq(e + 1) <= 1e-4:
        neg_eig_pairs.append((-1, sym_vecs[i]))
    else:
        eig_rotors.append(simple_rotor_sqrt(e))

sym_vecs, [[e.reverse().sw(a) for e in eig_rotors] for a in sym_vecs]

([-0.816 𝐞₁ + 0.091 𝐞₂ + 0.538 𝐞₃ + 0.19 𝐞₄,
  0.563 𝐞₁ + 0.236 𝐞₂ + 0.66 𝐞₃ + 0.438 𝐞₄,
  0.116 𝐞₁ + -0.0264 𝐞₂ + 0.486 𝐞₃ + -0.866 𝐞₄,
  0.0574 𝐞₁ + -0.967 𝐞₂ + 0.198 𝐞₃ + 0.148 𝐞₄],
 [[-0.816 𝐞₁ + 0.091 𝐞₂ + 0.538 𝐞₃ + 0.19 𝐞₄,
   -0.816 𝐞₁ + 0.091 𝐞₂ + 0.538 𝐞₃ + 0.19 𝐞₄,
   -0.816 𝐞₁ + 0.091 𝐞₂ + 0.538 𝐞₃ + 0.19 𝐞₄],
  [0.0313 𝐞₁ + -0.0609 𝐞₂ + 0.383 𝐞₃ + -0.921 𝐞₄,
   0.0313 𝐞₁ + -0.0609 𝐞₂ + 0.383 𝐞₃ + -0.921 𝐞₄,
   0.563 𝐞₁ + 0.236 𝐞₂ + 0.66 𝐞₃ + 0.438 𝐞₄],
  [-0.574 𝐞₁ + -0.229 𝐞₂ + -0.724 𝐞₃ + -0.306 𝐞₄,
   -0.574 𝐞₁ + -0.229 𝐞₂ + -0.724 𝐞₃ + -0.306 𝐞₄,
   0.116 𝐞₁ + -0.0264 𝐞₂ + 0.486 𝐞₃ + -0.866 𝐞₄],
  [0.0574 𝐞₁ + -0.967 𝐞₂ + 0.198 𝐞₃ + 0.148 𝐞₄,
   0.0574 𝐞₁ + -0.967 𝐞₂ + 0.198 𝐞₃ + 0.148 𝐞₄,
   0.0574 𝐞₁ + -0.967 𝐞₂ + 0.198 𝐞₃ + 0.148 𝐞₄]])

In [115]:
#5.25 I selected an example with -1 as an eigenvalue, 
# so we need an extra fix for the canonical form.
theta = reduce(alg.gp, eig_rotors[::2])
(
    ortho(x),
    theta.reverse().sw(x) - (2*sum(P(x, neg_eig_pairs[i][1]) for i in range(len(neg_eig_pairs))))
)


(-0.376 𝐞₁ + 0.439 𝐞₂ + 0.29 𝐞₃ + -1.02 𝐞₄ + 5.49e-18 𝐞₁₂₃ + 5.64e-18 𝐞₁₂₄ + -7.18e-17 𝐞₁₃₄ + -1.14e-17 𝐞₂₃₄,
 -0.376 𝐞₁ + 0.439 𝐞₂ + 0.29 𝐞₃ + -1.02 𝐞₄)

In [116]:
#5.28
blade_exp(F1) * blade_exp(F2), blade_exp(F2) * blade_exp(F1), 

(-0.13 + 0.303 𝐞₁₂ + 0.238 𝐞₁₃ + 0.699 𝐞₁₄ + 0.000358 𝐞₂₃ + 0.434 𝐞₂₄ + 0.191 𝐞₃₄ + 0.348 𝐞₁₂₃₄,
 -0.13 + 0.303 𝐞₁₂ + 0.238 𝐞₁₃ + 0.699 𝐞₁₄ + 0.000358 𝐞₂₃ + 0.434 𝐞₂₄ + 0.191 𝐞₃₄ + 0.348 𝐞₁₂₃₄)

In [117]:
#5.29
blade_exp_ = blade_exp(F1)
u1 = P(e1, F1).normalized()
u2 = u1 | blade_exp_
blade_exp_, u1 * u2

(0.936 + 0.0787 𝐞₁₂ + -0.16 𝐞₁₃ + 0.0141 𝐞₁₄ + 0.265 𝐞₂₃ + -0.0818 𝐞₂₄ + 0.119 𝐞₃₄,
 0.936 + 0.0787 𝐞₁₂ + -0.16 𝐞₁₃ + 0.0141 𝐞₁₄ + 0.265 𝐞₂₃ + -0.0818 𝐞₂₄ + 0.119 𝐞₃₄)

In [118]:
# reflection as sandwich product
x - 2 * P(x, neg_eig_pairs[0][1]), -neg_eig_pairs[0][1].sw(x)

(0.313 𝐞₁ + 0.784 𝐞₂ + 0.833 𝐞₃ + 0.235 𝐞₄,
 0.313 𝐞₁ + 0.784 𝐞₂ + 0.833 𝐞₃ + 0.235 𝐞₄)

In [119]:
#5.30
theta2 = theta * neg_eig_pairs[0][1]
ortho(x), -neg_eig_pairs[0][1].sw(theta.reverse().sw(x)), -theta2.reverse().sw(x)

(-0.376 𝐞₁ + 0.439 𝐞₂ + 0.29 𝐞₃ + -1.02 𝐞₄ + 5.49e-18 𝐞₁₂₃ + 5.64e-18 𝐞₁₂₄ + -7.18e-17 𝐞₁₃₄ + -1.14e-17 𝐞₂₃₄,
 -0.376 𝐞₁ + 0.439 𝐞₂ + 0.29 𝐞₃ + -1.02 𝐞₄,
 -0.376 𝐞₁ + 0.439 𝐞₂ + 0.29 𝐞₃ + -1.02 𝐞₄)

In [120]:
#5.37a
outermorphism(ortho, A3, alg), (-1)**(3*1)*theta2.reverse().sw(A3)

(0.198 𝐞₁₂₃ + -0.187 𝐞₁₂₄ + -0.949 𝐞₁₃₄ + -0.158 𝐞₂₃₄,
 0.198 𝐞₁₂₃ + -0.187 𝐞₁₂₄ + -0.949 𝐞₁₃₄ + -0.158 𝐞₂₃₄)

In [121]:
#5.37b
adjoint_outermorphism(ortho, F2, alg), (-1)**(2*1)*theta2.sw(F2)

(0.164 𝐞₁₂ + 0.768 𝐞₁₃ + 0.211 𝐞₁₄ + -0.799 𝐞₂₃ + 0.0525 𝐞₂₄ + 1.27 𝐞₃₄,
 0.164 𝐞₁₂ + 0.768 𝐞₁₃ + 0.211 𝐞₁₄ + -0.799 𝐞₂₃ + 0.0525 𝐞₂₄ + 1.27 𝐞₃₄)

In [122]:
#5.38
outermorphism(ortho, A3 * F2, alg), outermorphism(ortho, A3, alg) * outermorphism(ortho, F2, alg)

(0.287 𝐞₁ + -0.36 𝐞₂ + -1.12 𝐞₃ + -0.896 𝐞₄ + 0.226 𝐞₁₂₃ + 0.318 𝐞₁₂₄ + -0.132 𝐞₁₃₄ + 0.699 𝐞₂₃₄,
 0.287 𝐞₁ + -0.36 𝐞₂ + -1.12 𝐞₃ + -0.896 𝐞₄ + 0.226 𝐞₁₂₃ + 0.318 𝐞₁₂₄ + -0.132 𝐞₁₃₄ + 0.699 𝐞₂₃₄)

In [123]:
#5.39
from scipy.stats import ortho_group
matrix = ortho_group.rvs(dim=5)
alg = Algebra(5)
ortho = matrix2trans(matrix, alg)
x = create_r_vectors(1, alg)[0]
ortho_curl = curl(ortho, x, alg, grade=1)
ortho_curl ** 2

-2.41 + -1.11e-16 𝐞₁₂ + 1.91e-17 𝐞₁₅ + 0.129 𝐞₁₂₃₄ + 0.933 𝐞₁₂₃₅ + -0.101 𝐞₁₂₄₅ + -0.0177 𝐞₁₃₄₅ + -0.133 𝐞₂₃₄₅

In [124]:
# check orthogonality
x, ortho(adjoint_outermorphism(ortho, x, alg, h=1e-2))

(0.266 𝐞₁ + 0.167 𝐞₂ + 0.902 𝐞₃ + 0.491 𝐞₄ + 0.983 𝐞₅,
 0.266 𝐞₁ + 0.167 𝐞₂ + 0.902 𝐞₃ + 0.491 𝐞₄ + 0.983 𝐞₅)

In [125]:
# only a single vector left in the dual of skew blades, 
# so it's the remaining eigenvector for the symmetric part
# with eigenvalue 1 or -1: identity or a reflection

skew_blades = bivector_split(ortho_curl/2, alg)
sym_vec = reduce(alg.ip, skew_blades, alg.pseudoscalar((1,))).normalized()
p = 0
if max_diff(ortho(sym_vec), -sym_vec) < 1e-6:
    p += 1

In [126]:
rotors = []
for blade in skew_blades:
    t = P(sum(alg.frame), blade)
    # blade_norm = norm(blade)
    # we can't simply rely on ortho_m:
    # angle = np.arcsin(blade_norm) has no unique solution in [0, pi].
    # (pi - angle) generates the same sine, but differnt cosine (alpha).
    
    alpha = np.median(terms_ratio(ortho(t) - (t | ortho_curl)/2, t))
    # half_blade_ortho = (angle/blade_norm/2) * blade.reverse()
    # rotors.append(blade_exp(half_blade_ortho))
    rotors.append(simple_rotor_sqrt(alpha + blade))
rotor = reduce(alg.gp, rotors)
if p == 1:
    rotor2 = sym_vec * rotor
else:
    rotor2 = rotor
(-1) ** p * rotor2.reverse().sw(x), ortho(x)

(-1.1 𝐞₁ + 0.0251 𝐞₂ + -0.558 𝐞₃ + 0.739 𝐞₄ + 0.242 𝐞₅ + -5.47e-16 𝐞₁₂₃₄₅,
 -1.1 𝐞₁ + 0.0251 𝐞₂ + -0.558 𝐞₃ + 0.739 𝐞₄ + 0.242 𝐞₅)

In [127]:
# On the other hand, we can solve the rotor part by Lasenby's method
frame = [item for blade in skew_blades for item in blade_split(blade, alg)]
r_frame = reciprocal(frame)
lasenby_rotor = sum(char_multi_linear(ortho, s, alg, frame, r_frame) for s in range(5))
terms_ratio(rotor, lasenby_rotor)

array([0.87163893, 0.87163893, 0.87163893, 0.87163893, 0.87163893,
       0.87163893, 0.87163893, 0.87163893, 0.87163893, 0.87163893,
       0.87163893, 0.87163893, 0.87163893, 0.87163893, 0.87163893,
       0.87163893])

In [128]:
# For orthogonal transform, we simply normalize it
l_rotor = lasenby_rotor/norm(lasenby_rotor)
if p == 1:
    l_rotor = l_rotor * sym_vec
(-1) ** p * l_rotor.reverse().sw(x), ortho(x)

(-1.1 𝐞₁ + 0.0251 𝐞₂ + -0.558 𝐞₃ + 0.739 𝐞₄ + 0.242 𝐞₅ + -4.62e-16 𝐞₁₂₃₄₅,
 -1.1 𝐞₁ + 0.0251 𝐞₂ + -0.558 𝐞₃ + 0.739 𝐞₄ + 0.242 𝐞₅)